# Feature Engineering for our House Prediction Dataset
1. Missing Values
2. Temporal Variables
3. Categorical Variables
4. Standardize values of Variables to same range

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [84]:
df = pd.read_csv("../data/train.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


There is a chance of data leakage, hence we first split the data and then apply feature engineering

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['SalePrice'], test_size=0.1, random_state=0)

In [86]:
X_train.shape, X_test.shape

((1314, 81), (146, 81))

##### Treating the missing values

In [87]:
nan_feat = [feature for feature in df.columns if df[feature].isnull().sum() > 1 and df[feature].dtypes == 'O']
for feat in nan_feat:
    print("{}: {}% missing values".format(feat, np.round(df[feat].isnull().mean() * 100), 4))


Alley: 94.0% missing values
MasVnrType: 60.0% missing values
BsmtQual: 3.0% missing values
BsmtCond: 3.0% missing values
BsmtExposure: 3.0% missing values
BsmtFinType1: 3.0% missing values
BsmtFinType2: 3.0% missing values
FireplaceQu: 47.0% missing values
GarageType: 6.0% missing values
GarageFinish: 6.0% missing values
GarageQual: 6.0% missing values
GarageCond: 6.0% missing values
PoolQC: 100.0% missing values
Fence: 81.0% missing values
MiscFeature: 96.0% missing values


In [88]:
def replace_cat_feat(df, nan_feat):
    dta = df.copy()
    dta[nan_feat] = dta[nan_feat].fillna('Missing')
    return dta

df = replace_cat_feat(df, nan_feat)
df[nan_feat].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [89]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,...,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,...,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,...,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,...,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,...,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000


In [90]:
num_na_feat = [feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']
for feat in num_na_feat:
    print("{}: {}% of missing numerical features".format(feat, np.around(df[feat].isnull().mean() * 100, 4)))

LotFrontage: 17.7397% of missing numerical features
MasVnrArea: 0.5479% of missing numerical features
GarageYrBlt: 5.5479% of missing numerical features


In [91]:
for feat in num_na_feat:
    med_val = df[feat].median()
    df[feat+'_nan'] = np.where(df[feat].isnull(), 1, 0)
    df[feat].fillna(med_val, inplace=True)
    
df[num_na_feat].isnull().sum()

/var/folders/y5/zjrfwy6d3992b9fgml0yq5700000gn/T/ipykernel_56117/96177141.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feat].fillna(med_val, inplace=True)
/var/folders/y5/zjrfwy6d3992b9fgml0yq5700000gn/T/ipykernel_56117/96177141.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [92]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,2,2008,WD,Normal,208500,0,0,0
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,9,2008,WD,Normal,223500,0,0,0
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,2,2006,WD,Abnorml,140000,0,0,0
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,12,2008,WD,Normal,250000,0,0,0


In [94]:
for feat in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
    df[feat] = df['YrSold'] - df[feat]
df[['YearBuilt', 'YearRemodAdd', "GarageYrBlt"]].head()

,YearBuilt,YearRemodAdd,GarageYrBlt
0,2003,2003,2003.0
1,1976,1976,1976.0
2,2001,2002,2001.0
3,1915,1970,1998.0
4,2000,2000,2000.0


We now calculate log of numerical feature in-order to optimise the values

In [97]:
num_feat = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for feat in num_feat:
    df[feat] = np.log(df[feat])

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,RL,0.356952,0.789308,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,2,2008,WD,Normal,0.918424,0,0,0
1,2,20,RL,0.390359,0.795652,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,5,2007,WD,Normal,0.913868,0,0,0
2,3,60,RL,0.364448,0.803361,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,9,2008,WD,Normal,0.920679,0,0,0
3,4,70,RL,0.343311,0.795395,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,2,2006,WD,Abnorml,0.905140,0,0,0
4,5,60,RL,0.397825,0.814538,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,12,2008,WD,Normal,0.924279,0,0,0


Here we find and mark rare categorical feature that can basically reduce performance of our ML algo due to high cardinality (or too many categories) and reduce overfitting

In [101]:
cat_feat = [feat for feat in df.columns if df[feat].dtype == 'O']
print(cat_feat)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [109]:
for feat in cat_feat:
    temp = df.groupby(feat)['SalePrice'].count()/len(df)
    temp_df = temp[temp > 0.01].index
    df[feat] = np.where(df[feat].isin(temp_df), df[feat], 'Rare_var')

In [117]:
df.head(100)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,RL,0.356952,0.789308,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,2,2008,WD,Normal,0.918424,0,0,0
1,2,20,RL,0.390359,0.795652,Pave,Missing,Reg,Lvl,AllPub,...,Missing,0,5,2007,WD,Normal,0.913868,0,0,0
2,3,60,RL,0.364448,0.803361,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,9,2008,WD,Normal,0.920679,0,0,0
3,4,70,RL,0.343311,0.795395,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,2,2006,WD,Abnorml,0.905140,0,0,0
4,5,60,RL,0.397825,0.814538,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,12,2008,WD,Normal,0.924279,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,60,RL,0.366844,0.796489,Pave,Missing,IR2,Lvl,AllPub,...,Shed,480,4,2009,WD,Normal,0.914500,1,0,0
96,97,20,RL,0.386430,0.798926,Pave,Missing,IR1,Lvl,AllPub,...,Missing,0,8,2006,WD,Normal,0.919271,0,0,0
97,98,20,RL,0.375963,0.801933,Pave,Missing,Reg,HLS,AllPub,...,Missing,0,5,2007,WD,Normal,0.891496,0,0,0
98,99,30,RL,0.399616,0.800605,Pave,Missing,Reg,Lvl,AllPub,...,Shed,400,5,2010,COD,Abnorml,0.886719,0,0,0


In [120]:
for feat in cat_feat:
    labels_ordered = df.groupby([feat])['SalePrice'].mean().sort_values().index
    labels_ordered = {k : i for i, k in enumerate(labels_ordered, 0)}
    df[feat] = df[feat].map(labels_ordered)

In [121]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,3,0.356952,0.789308,1,2,0,1,1,...,2,0,2,2008,2,3,0.918424,0,0,0
1,2,20,3,0.390359,0.795652,1,2,0,1,1,...,2,0,5,2007,2,3,0.913868,0,0,0
2,3,60,3,0.364448,0.803361,1,2,1,1,1,...,2,0,9,2008,2,3,0.920679,0,0,0
3,4,70,3,0.343311,0.795395,1,2,1,1,1,...,2,0,2,2006,2,0,0.905140,0,0,0
4,5,60,3,0.397825,0.814538,1,2,1,1,1,...,2,0,12,2008,2,3,0.924279,0,0,0


##### Feature scaling

In [123]:
feat_scale = [feat for feat in df.columns if feat not in ['Id', 'SalePrice']]
len(feat_scale)

82

In [124]:
feat_scale

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'Enc

In [125]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,60,3,0.356952,0.789308,1,2,0,1,1,...,2,0,2,2008,2,3,0.918424,0,0,0
1,2,20,3,0.390359,0.795652,1,2,0,1,1,...,2,0,5,2007,2,3,0.913868,0,0,0
2,3,60,3,0.364448,0.803361,1,2,1,1,1,...,2,0,9,2008,2,3,0.920679,0,0,0
3,4,70,3,0.343311,0.795395,1,2,1,1,1,...,2,0,2,2006,2,0,0.905140,0,0,0
4,5,60,3,0.397825,0.814538,1,2,1,1,1,...,2,0,12,2008,2,3,0.924279,0,0,0


In [127]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[feat_scale])

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [129]:
scaler.transform(df[feat_scale])

array([[0.23529412, 0.75      , 0.55289564, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.62690092, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.75      , 0.56949989, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.29411765, 0.75      , 0.55854802, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.56949989, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.60454218, ..., 0.        , 0.        ,
        0.        ]], shape=(1460, 82))

In [131]:
dta = pd.concat([df[['Id', 'SalePrice']].reset_index(drop = True), pd.DataFrame(scaler.transform(df[feat_scale]), columns = feat_scale)], axis = 1)

In [133]:
dta.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,0.918424,0.235294,0.75,0.552896,0.460993,1.0,1.0,0.000000,0.333333,...,0.75,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,0.913868,0.000000,0.75,0.626901,0.487266,1.0,1.0,0.000000,0.333333,...,0.75,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,0.920679,0.235294,0.75,0.569500,0.519193,1.0,1.0,0.333333,0.333333,...,0.75,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,0.905140,0.294118,0.75,0.522676,0.486201,1.0,1.0,0.333333,0.333333,...,0.75,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,0.924279,0.235294,0.75,0.643440,0.565482,1.0,1.0,0.333333,0.333333,...,0.75,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [134]:
dta.to_csv('X_train.csv', index = False)